In [ ]:
from common import *
import matplotlib.dates as mdates

## Snobal 

In [ ]:
snobal_dir = f'{Path.home()}/scratch/iSnobal/Snotel/2x2-grid'
coarsen_opts = dict(x=2, y=2, keep_attrs=True)

schofield_snobal = xr.open_dataset(f'{snobal_dir}/2018-Schofield-snobal.zarr')
butte_snobal = xr.open_dataset(f'{snobal_dir}/2018-Butte-snobal.zarr')
taylor_snobal = xr.open_dataset(f'{snobal_dir}/2018-Taylor-snobal.zarr')

## SNOTEL

In [ ]:
csv_parser_opts = dict(
    comment='#',
    parse_dates=True,
    index_col=0,
    names=['Date', 'Depth(m)', 'SWE(mm)', 'Density(kg/m3)', 'Wind(km/h)', 'Air-T(C)'],
    header=0,
)

schofield_snotel = pd.read_csv(
    f'{Path.home()}/scratch/iSnobal/Snotel/2018-Schofield-Snotel.csv',
    **csv_parser_opts,
)
schofield_snotel['Depth(m)'] = schofield_snotel['Depth(m)'] / 100
schofield_snotel['Density(kg/m3)'] = schofield_snotel['Density(kg/m3)'] * 10

butte_snotel = pd.read_csv(
    f'{Path.home()}/scratch/iSnobal/Snotel/2018-Butte-Snotel.csv',
    **csv_parser_opts,
)
butte_snotel['Depth(m)'] = butte_snotel['Depth(m)'] / 100
butte_snotel['Density(kg/m3)'] = butte_snotel['Density(kg/m3)'] * 10

taylor_snotel = pd.read_csv(
    f'{Path.home()}/scratch/iSnobal/Snotel/2018-Upper-Taylor-Snotel.csv',
    **csv_parser_opts,
)
taylor_snotel['Depth(m)'] = taylor_snotel['Depth(m)'] / 100
taylor_snotel['Density(kg/m3)'] = taylor_snotel['Density(kg/m3)'] * 10

## Plot data

In [ ]:
plot_data = {
    'Butte': {
        'min': butte_snobal.coarsen(**coarsen_opts).min(),
        'max': butte_snobal.coarsen(**coarsen_opts).max(),
        'snotel': butte_snotel,
    },
    'Schofield Pass': {
        'min': schofield_snobal.coarsen(**coarsen_opts).min(),
        'max': schofield_snobal.coarsen(**coarsen_opts).max(),
        'snotel': schofield_snotel,
    },
    'Upper Taylor': {
        'min': taylor_snobal.coarsen(**coarsen_opts).min(),
        'max': taylor_snobal.coarsen(**coarsen_opts).max(),
        'snotel': taylor_snotel,
    },
}

In [ ]:
figure_opts = dict(figsize=(6,4), dpi=300, tight_layout=True)
water_year = pd.date_range(start='2017-10-01', periods=11, freq='MS')
text_props = dict(facecolor='whitesmoke', alpha=0.5, pad=.5, boxstyle='round')

xTicks = mdates.DateFormatter('%d-%b')

def plot_variable(snobal, snotel, title, label):
    for site in plot_data.keys():
        figure, ax = plt.subplots(1, 1, **figure_opts)
        figure.set_facecolor('lightgrey')
        
        plot_data[site]['min'][snobal].plot(ax=ax, c='slategrey', alpha=0.8)
        plot_data[site]['max'][snobal].plot(ax=ax, c='slategrey', alpha=0.8)
        ax.fill_between(
            plot_data[site]['min'].time, 
            plot_data[site]['min'][snobal].data.flatten(), 
            plot_data[site]['max'][snobal].data.flatten(),
            label='Snobal',
            color='navajowhite', alpha=0.8
        )
        plot_data[site]['snotel'][snotel].plot(ax=ax, c='steelblue', label='Snotel')
        
        ax.text(
            0.03, 0.95, 
            site,
            style='italic',
            transform=ax.transAxes, 
            verticalalignment='top', 
            bbox=text_props
        )
        
        ax.legend(loc='upper right');
        ax.xaxis.set_major_formatter(xTicks)
        ax.set_xlim([water_year[0], water_year[-1]])
        ax.set_xlabel('Water Year 2018')
        ax.set_ylim(bottom=0)
        ax.set_ylabel(label)
        ax.set_title(None);
        
        filename = f'{title}_{site}'.replace(' ', '-')
        plt.savefig(f"/data/iSnobal/figures/{filename}.png")
        

## Comparison stats 

In [ ]:
for site in plot_data.keys():
    snobal_min = int(plot_data[site]['min']['specific_mass'].sum())
    snobal_max = int(plot_data[site]['max']['specific_mass'].sum())
    snotel = int(plot_data[site]['snotel']['SWE(mm)'].sum())

    print(f'{site}:')
    print(f" Snobal max: {snobal_max} mm")
    print(f" Snobal min: {snobal_min} mm")
    print(f" Snotel: {snotel} mm")
    print(f" Snotel % min: {snobal_min/snotel:.2%}")
    print(f" Snotel % max: {snobal_max/snotel:.2%}")

In [ ]:
melt_start = pd.to_datetime('2018-04-01')

for site in plot_data.keys():
    print(site)
    print('Snobal:')
    snobal_depths = plot_data[site]['min']['thickness'].squeeze('x').squeeze('y').to_pandas()
    min_date = snobal_depths[(snobal_depths == 0) & (snobal_depths.index > melt_start)].index[0]
    print(min_date)
    snobal_depths = plot_data[site]['max']['thickness'].squeeze('x').squeeze('y').to_pandas()
    max_date = snobal_depths[(snobal_depths == 0) & (snobal_depths.index > melt_start)].index[0]
    print(max_date)
    
    snotel_depth = plot_data[site]['snotel']['Depth(m)']
    a = snotel_depth[(snotel_depth == 0) & (snotel_depth.index > melt_start)].index[0]
    print('Snotel:')
    print(a)
    print('Difference:')
    print(f' Min: {min_date - a}')
    print(f' Max: {max_date - a}')
    print('----')

## SWE 

In [ ]:
plot_variable('specific_mass', 'SWE(mm)', 'SWE', 'SWE (mm)')

## Depth

In [ ]:
plot_variable('thickness', 'Depth(m)', 'Snow Depth', 'Snow Depth (m)')

## Snow Density

In [ ]:
plot_variable('snow_density', 'Density(kg/m3)', 'Snow Density', r'Snow Density ($kg/m^3$)')

## Temperature 

In [ ]:
plot_variable('temp_surf', 'Air-T(C)', 'Temperature', 'Temperatreu (C)')